In [4]:
order_products_train = pd.read_csv("order_products__train.csv")
order_products_prior = pd.read_csv("order_products__prior.csv")
orders = pd.read_csv("orders.csv")
products = pd.read_csv("products.csv")
aisles = pd.read_csv("aisles.csv")
departments = pd.read_csv("departments.csv")
submissions = pd.read_csv("sample_submission.csv")

## FEATURE ENGINEERING

In [8]:
''' 
we have seen from the graph that how user purchase pattern change  within a day as well as period of month. 
So, we thought to use this as feature .

'''
orders['reorder_pattern'] = 'daily'
orders['reorder_pattern'][orders['days_since_prior_order'] <= 1] = 'daily'
orders['reorder_pattern'][(orders['days_since_prior_order'] > 1) & (orders['days_since_prior_order'] <= 7)]  = 'weekly'
orders['reorder_pattern'][(orders['days_since_prior_order'] > 7) & (orders['days_since_prior_order'] <= 15)]  = 'biweekly'
orders['reorder_pattern'][(orders['days_since_prior_order'] > 15) & (orders['days_since_prior_order'] <= 30)]  = 'monthly'

# Set a default value
orders['name_of_day_time'] = 'Morning'
orders['name_of_day_time'][(orders['order_hour_of_day'] > 12) & (orders['order_hour_of_day'] <= 17)]  = 'Afternoon'
orders['name_of_day_time'][(orders['order_hour_of_day'] > 17) & (orders['order_hour_of_day'] <= 19)]  = 'evening'
orders['name_of_day_time'][(orders['order_hour_of_day'] > 19) & (orders['order_hour_of_day'] <= 24)]  = 'night'
orders['name_of_day_time'][(orders['order_hour_of_day'] > 0) & (orders['order_hour_of_day'] <= 4)]  = 'night'

In [ ]:
# read the prior order file #
prior_df = pd.read_csv("order_products__prior.csv")
prior_df1=prior_df[(prior_df['reordered'] == 1)]

# merge with the orders file to get the user_id #
prior_df1 = pd.merge(prior_df1, orders, how="inner", on="order_id")
prior_df1.head()

In [24]:
'''
 reading the prior data of users ordering products where reordering status is 1 beacuse we are all concern about reorder purchasing items.
 
''' 
prior_ = pd.read_csv("order_products__prior.csv")
prior_data=prior_[prior_.reordered == 1]

# merge  the orders data with prior_data on order_id
prior_data = pd.merge(prior_data, orders, how="inner", on="order_id")
prior_data.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,reorder_pattern,name_of_day_time
0,2,33120,1,1,202279,prior,3,5,9,8.0,biweekly,Morning
1,2,28985,2,1,202279,prior,3,5,9,8.0,biweekly,Morning
2,2,45918,4,1,202279,prior,3,5,9,8.0,biweekly,Morning
3,2,17794,6,1,202279,prior,3,5,9,8.0,biweekly,Morning
4,2,40141,7,1,202279,prior,3,5,9,8.0,biweekly,Morning


In [10]:
''' 
since i am planning to use last 5 purchase item history as feature to boost the model so i am avoiding the train and test evals set.

'''
train_orders=orders[orders.eval_set != 'test']

train_orders=train_orders[train_orders.eval_set != 'train']

prior_past_data = pd.merge(order_products_prior, train_orders, how="inner", on="order_id")


reorder_rank = prior_past_data[['user_id','order_number']]
reorder_rank = reorder_rank.sort_values(by ='user_id')
reorder_rank = reorder_rank.drop_duplicates()
reorder_rank['rank'] = reorder_rank.groupby(['user_id'])['order_number'].rank(ascending=False,method='dense')

#creating function to extract user prior history
def recent_ordering_status(rank):
  reorder_history = reorder_rank[(reorder_rank['rank'] == 1)]
  data = pd.merge(prior_past_data, reorder_history, how="inner", on=["user_id", "order_number"])
  data = data[['user_id','product_id','add_to_cart_order','days_since_prior_order','order_hour_of_day','reordered']]
  data['time_in_sec'] = data['days_since_prior_order']*24*60*60 + data['order_hour_of_day']*60*60
  data = data[["user_id", "product_id","add_to_cart_order" ,"time_in_sec","reordered"]]
  data = data.sort_values(by ='user_id')
  data.columns = ["user_id", "product_id","add_to_cart_order"+str(rank) ,"time_in_sec"+str(rank),"reordered_latest"+str(rank)]

  return data


In [ ]:
# storing the last 5 ordering history
data1 = recent_ordering_status(1)
data2 = recent_ordering_status(2)
data3 = recent_ordering_status(3)
data4 = recent_ordering_status(4)
data5 = recent_ordering_status(5)

In [25]:
'''  
since our problem statement is to predict which of the previously purchase item will be in user next order. so we need to know
first which are the products that user had purchased in past and then we merge it with train data with class label 1 as reorder item and 0 
as not reorder item.

'''

use_prod = prior_data[["order_id", "product_id"]]
ors  =  orders[['order_id','user_id']]
use_prod1 = pd.merge(use_prod, ors, how="inner", on='order_id')
use_prod1 = use_prod1[['user_id','product_id']]
use_prod1 = use_prod1.drop_duplicates()

In [ ]:
# we are finding the number of times the particular product has been reorder by the user in past.
reorder_count = prior_data.groupby(["user_id","product_id"])["reordered"].aggregate(["count"]).reset_index()
reorder_count.columns = ["user_id", "product_id", "reordered_count"]

# we are finding the number of times the particular product has been reorder in past.
reorder_count_by_product = prior_data.groupby("product_id")["reordered"].aggregate(["count"]).reset_index()
reorder_count_by_product.columns = ["product_id", "product_reordered_count"]

In [26]:
#order id  on which we have to perform training
train_data = pd.read_csv("order_products__train.csv", usecols=["order_id"]).drop_duplicates()
# merging trainig order id with order data
train_data = pd.merge(train_data, orders, how="inner", on="order_id")
train_data = pd.merge(train_data, use_prod1, how="inner", on="user_id")
train_data.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,reorder_pattern,name_of_day_time,product_id
0,1,112108,train,4,4,10,9.0,biweekly,Morning,14947
1,1,112108,train,4,4,10,9.0,biweekly,Morning,5707
2,1,112108,train,4,4,10,9.0,biweekly,Morning,30881
3,1,112108,train,4,4,10,9.0,biweekly,Morning,43633
4,1,112108,train,4,4,10,9.0,biweekly,Morning,49302


In [27]:
#order id  on which we have to predict product which could be in user next order
test_data = pd.read_csv("sample_submission.csv", usecols=["order_id"])
# merging trainig order id with order data
test_data = pd.merge(test_data, orders, how="inner", on="order_id")
test_data = pd.merge(test_data, use_prod1, how="inner", on="user_id")
test_data.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,reorder_pattern,name_of_day_time,product_id
0,17,36855,test,5,6,15,1.0,daily,Afternoon,21709
1,17,36855,test,5,6,15,1.0,daily,Afternoon,47766
2,17,36855,test,5,6,15,1.0,daily,Afternoon,13107
3,17,36855,test,5,6,15,1.0,daily,Afternoon,21463
4,17,36855,test,5,6,15,1.0,daily,Afternoon,38777


In [28]:
products_ = pd.read_csv("products.csv", usecols=["product_id", "aisle_id", "department_id"])
products_.head()

,product_id,aisle_id,department_id
0,1,61,19
1,2,104,13
2,3,94,7
3,4,38,1
4,5,5,13


In [ ]:
# addiing aisles and depertment id to train and test data
train_data = pd.merge(train_data, products_, how="inner", on="product_id")
test_data = pd.merge(test_data, products_, how="inner", on="product_id")

In [ ]:
#merging the last 5 orderig history to train and test data to boost the model
train_data = pd.merge(train_data, data5, how="left", on=['user_id',"product_id"])
train_data = pd.merge(train_data, data4, how="left", on=['user_id',"product_id"])
train_data = pd.merge(train_data, data3, how="left", on=['user_id',"product_id"])
train_data = pd.merge(train_data, data2, how="left", on=['user_id',"product_id"])
train_data = pd.merge(train_data, data1, how="left", on=['user_id',"product_id"])

test_data = pd.merge(test_data, data5, how="left", on=['user_id',"product_id"])
test_data = pd.merge(test_data, data4, how="left", on=['user_id',"product_id"])
test_data = pd.merge(test_data, data3, how="left", on=['user_id',"product_id"])
test_data = pd.merge(test_data, data2, how="left", on=['user_id',"product_id"])
test_data = pd.merge(test_data, data1, how="left", on=['user_id',"product_id"])

In [ ]:
# addiing  depertment detail to train and test data
train_data = pd.merge(train_data, departments, how="left", on="department_id")
test_data = pd.merge(test_data, departments, how="left", on="department_id")

# addiing  aisles detail to train and test data
train_data = pd.merge(train_data, aisles, how="left", on="aisle_id")
test_data = pd.merge(test_data, aisles, how="left", on="aisle_id")

# addiing  reorder_count detail to train and test data
train_data = pd.merge(train_data, reorder_count, how="left", on=['user_id',"product_id"])
test_data = pd.merge(test_data, reorder_count, how="left", on=['user_id',"product_id"])

# addiing  reorder_count_by_product detail to train and test data
train_data = pd.merge(train_data, reorder_count_by_product, how="left", on="product_id")
test_data = pd.merge(test_data, reorder_count_by_product, how="left", on="product_id")


In [ ]:
'''
as we have seen from the graph that there are some product which has be order so many times by user as well as there
are some department and aisle name as well from which product has been be taken from frequently. so, i thougth to consider 
this as feature in my model built.

'''

product_count = pd.merge(prior_data, products, how="inner", on="product_id")
k = product_count['product_name'].value_counts()

k = dict(k) 
product_count_dict = {i:j/len(product_count) for i , j  in k.items()}

####################

aisles_count = pd.merge(prior_data, products, how="inner", on="product_id")
aisles_count = pd.merge(aisles_count, aisles, how="inner", on="aisle_id")
k = aisles_count['aisle'].value_counts()

k = dict(k) 
aisles_count_dict = {i:j/len(aisles_count) for i , j  in k.items()}

###################

department_count = pd.merge(prior_data, products, how="inner", on="product_id")
department_count = pd.merge(department_count, aisles, how="inner", on="aisle_id")
department_count = pd.merge(department_count, departments, how="inner", on="department_id")
k = department_count['department'].value_counts()

k = dict(k) 
department_count_dict = {i:j/len(department_count) for i , j  in k.items()}

In [ ]:
#merging product count ratio with train data
final_product_name = train_data['product_name'].values
k = []
for i in final_product_name:
  if i in product_count_dict:
    k.append(product_count_dict[i])
  else:
    k.append(0) 
final_product_name = k     
train_data['product_name_ratio'] = final_product_name

#merging aisles count ratio with train data
final_aisle = train_data['aisle'].values
k = []
for i in final_aisle:
  if i in aisles_count_dict:
    k.append(aisles_count_dict[i])
  else:
    k.append(0) 
final_aisle = k 
train_data['aisle_ratio'] = final_aisle

#merging department count ratio with train data
final_department = train_data['department'].values
k = []
for i in final_department:
  if i in department_count_dict:
    k.append(department_count_dict[i])
  else:
    k.append(0) 
final_department = k  
train_data['department_ratio'] = final_department

#############################################################

#merging product count ratio with test data
final_product_name = test_data['product_name'].values
k = []
for i in final_product_name:
  if i in product_count_dict:
    k.append(product_count_dict[i])
  else:
    k.append(0) 
final_product_name = k  
test_data['product_name_ratio'] = final_product_name

#merging aisle count ratio with test data
final_aisle = test_data['aisle'].values
k = []
for i in final_aisle:
  if i in aisles_count_dict:
    k.append(aisles_count_dict[i])
  else:
    k.append(0) 
final_aisle = k 
test_data['aisle_ratio'] = final_aisle

#merging department count ratio with test data
final_department = test_data['department'].values
k = []
for i in final_department:
  if i in department_count_dict:
    k.append(department_count_dict[i])
  else:
    k.append(0) 
final_department = k 
test_data['department_ratio'] = final_department

In [ ]:
'''
in this section i am basically taking about order_dow , order_hour_of_day, days_since_prior_order , name_of_day_time
and reorder_pattern . User purchasing product is proportion to above given attibutes. so i am counting the no of times
user purchase item in given tie period as well as giving weightage to the feature as well .

'''



all_train_order=orders[(orders['eval_set'] != 'test')]
all_train_order=orders[(orders['eval_set'] != 'train')]

k =all_train_order['order_dow'].value_counts()
k = dict(k) 
dow_dict = {i:j/len(orders) for i , j  in k.items()}

#######################################

j =all_train_order['order_hour_of_day'].value_counts()
j = dict(j)
hour_of_day = {i:j/len(orders) for i , j  in j.items()}

#########################################

k = all_train_order['days_since_prior_order'].value_counts()
k = dict(k)
days_since_prior = {i:j/len(orders) for i , j  in k.items()}

##########################################

k = all_train_order['name_of_day_time'].value_counts()
k = dict(k)
new_name_of_day_time = {i:j/len(orders) for i , j  in k.items()}

##########################################

k = all_train_order['reorder_pattern'].value_counts()
k = dict(k)
order_reorder_pattern = {i:j/len(orders) for i , j  in k.items()}

In [ ]:
#merging dow count ratio with train data
final_dow = train_data['order_dow'].values
final_dow = [dow_dict[i] for i in final_dow]
train_data['dow_ratio'] = final_dow

#merging hour_of_day count ratio with train data
final_order_hour_of_day = train_data['order_hour_of_day'].values
final_order_hour_of_day = [hour_of_day[i] for i in final_order_hour_of_day]
train_data['hour_of_day_ratio'] = final_order_hour_of_day

#merging days_since_prior count ratio with train data
final_days_since_prior = train_data['days_since_prior_order'].values
final_days_since_prior = [days_since_prior[i] for i in final_days_since_prior]
train_df['days_since_prior_ratio'] = final_days_since_prior

#merging reorder_pattern count ratio with train data
final_reorder_pattern = train_df['reorder_pattern'].values
final_reorder_pattern = [order_reorder_pattern[i] for i in final_reorder_pattern]
train_data['reorder_pattern_ratio'] = final_reorder_pattern

#merging new_name_of_day_time count ratio with train data
final_new_name_of_day_time = train_data['name_of_day_time'].values
final_new_name_of_day_time = [new_name_of_day_time[i] for i in final_new_name_of_day_time]
train_data['new_name_of_day_time_ratio'] = final_new_name_of_day_time

#merging dow count ratio with test data
final_dow = test_data['order_dow'].values
final_dow = [dow_dict[i] for i in final_dow]
test_data['dow_ratio'] = final_dow

#merging hour_of_day count ratio with test data
final_order_hour_of_day = test_data['order_hour_of_day'].values
final_order_hour_of_day = [hour_of_day[i] for i in final_order_hour_of_day]
test_data['hour_of_day_ratio'] = final_order_hour_of_day

#merging days_since_prior count ratio with test data
final_days_since_prior = test_data['days_since_prior_order'].values
final_days_since_prior = [days_since_prior[i] for i in final_days_since_prior]
test_data['days_since_prior_ratio'] = final_days_since_prior

#merging reorder_pattern count ratio with test data
final_reorder_pattern = test_data['reorder_pattern'].values
final_reorder_pattern = [order_reorder_pattern[i] for i in final_reorder_pattern]
test_data['reorder_pattern_ratio'] = final_reorder_pattern

#merging new_name_of_day_time count ratio with test data
final_new_name_of_day_time = test_data['name_of_day_time'].values
final_new_name_of_day_time = [new_name_of_day_time[i] for i in final_new_name_of_day_time]
test_data['new_name_of_day_time_ratio'] = final_new_name_of_day_time

In [ ]:
'''
since we are considering order_dow, order_hour_of_day, days_since_prior_order as important features . i am creating dow_by_user, 
order_hour_of_day_by_user and new_days_since_prior_order_by_user as feature adding to training data.

dow_by_user := it indicates how many times particular user do shopping in given day of week.

order_hour_of_day_by_user := it indicates how many times particular user do shopping in given hour of days.

new_days_since_prior_order_by_user := it indicates how many times particular user do shopping in given period of month.

'''

all_train_order=orders[(orders['eval_set'] != 'test')]
all_train_order=orders[(orders['eval_set'] != 'train')]

order_dow = all_train_order[['user_id','order_dow']]
new_dow = all_train_order['order_dow'].values
order_dow['new_dow'] = new_dow
order_dow = order_dow.groupby(["user_id",'order_dow'])["new_dow"].aggregate("count").reset_index()
order_dow.columns = ["user_id", "order_dow", "dow_by_user"]

##############################################################################


order_hour_of_day_by_user = all_train_order[['user_id','order_hour_of_day']]
new_order_hour_of_day = all_train_order['order_hour_of_day'].values
order_hour_of_day_by_user['new_order_hour_of_day'] = new_order_hour_of_day
order_hour_of_day_by_user = order_hour_of_day_by_user.groupby(["user_id",'order_hour_of_day'])["new_order_hour_of_day"].aggregate("count").reset_index()
order_hour_of_day_by_user.columns = ["user_id", "order_hour_of_day", "order_hour_of_day_by_user"]

################################################################################

days_since_prior_order_by_user = all_train_order[['user_id','days_since_prior_order']]
new_days_since_prior_order = all_train_order['days_since_prior_order'].values
days_since_prior_order_by_user['new_days_since_prior_order'] = new_days_since_prior_order
days_since_prior_order_by_user = days_since_prior_order_by_user.groupby(["user_id",'days_since_prior_order'])["new_days_since_prior_order"].aggregate("count").reset_index()
days_since_prior_order_by_user.columns = ["user_id", "days_since_prior_order", "new_days_since_prior_order_by_user"]

In [ ]:
#merging order_dow with train data
train_data = pd.merge(train_data, order_dow, how="left", on=['user_id',"order_dow"])

#merging order_hour_of_day_by_user with train data
train_data = pd.merge(train_data, order_hour_of_day_by_user, how="left", on=["user_id",'order_hour_of_day'])

#merging days_since_prior_order_by_user with train data
train_data = pd.merge(train_data, days_since_prior_order_by_user, how="left", on=["user_id",'days_since_prior_order'])

#merging order_dow with test data
test_data = pd.merge(test_data, order_dow, how="left", on=['user_id',"order_dow"])

#merging order_hour_of_day_by_user with test data
test_data = pd.merge(test_data, order_hour_of_day_by_user, how="left", on=["user_id",'order_hour_of_day'])

#merging days_since_prior_order_by_user with test data
test_data = pd.merge(test_data, days_since_prior_order_by_user, how="left", on=["user_id",'days_since_prior_order'])

In [ ]:
'''
same as above , i am doing further with some constrain . The constrains are user_id , product_id, order_dow, order_hour_of_day and
days_since_prior_order . 

new_prouse_order_dow :=  it indicates how many times particular user purchase particular items in given day of week.

new_order_hour_of_day := it indicates how many times particular user purchase particular items in given hour of day.

new_days_since_prior_order := it indicates how many times particular user purchase particular items in given period of month.

new_product_count_y :=  it indicates how many times particular user purchase particular items in given period of day and given day of week.

new_product_count_z :=  it indicates how many times particular user purchase particular items in given period of day ,  given day of week 
and period of month.

'''

new_products__prior = pd.merge(prior_data, products, how="inner", on="product_id")
new_products__prior = pd.merge(new_products__prior, departments, how="inner", on="department_id")
new_products__prior = pd.merge(new_products__prior, aisles, how="inner", on="aisle_id")

order_dow_by_prouse = new_products__prior[['user_id','product_id','order_dow']]
new_prouse_order_dow = new_products__prior['order_dow'].values
order_dow_by_prouse['new_prouse_order_dow'] = new_prouse_order_dow
order_dow_by_prouse = order_dow_by_prouse.groupby(['user_id','product_id','order_dow'])["new_prouse_order_dow"].aggregate("count").reset_index()
order_dow_by_prouse.columns = ['user_id','product_id','order_dow', 'new_prouse_order_dow']

###########################################################################################

order_hour_of_day_by_prouse = new_products__prior[['user_id','product_id','order_hour_of_day']]
new_order_hour_of_day = new_products__prior['order_hour_of_day'].values
order_hour_of_day_by_prouse['new_order_hour_of_day'] = new_order_hour_of_day
order_hour_of_day_by_prouse = order_hour_of_day_by_prouse.groupby(['user_id','product_id','order_hour_of_day'])["new_order_hour_of_day"].aggregate("count").reset_index()
order_hour_of_day_by_prouse.columns = ['user_id','product_id','order_hour_of_day', 'new_order_hour_of_day']

##############################################################################################

days_since_prior_order_by_prouse = new_products__prior[['user_id','product_id','days_since_prior_order']]
new_days_since_prior_order = new_products__prior['days_since_prior_order'].values
days_since_prior_order_by_prouse['new_days_since_prior_order'] = new_days_since_prior_order
days_since_prior_order_by_prouse = days_since_prior_order_by_prouse.groupby(['user_id','product_id','days_since_prior_order'])["new_days_since_prior_order"].aggregate("count").reset_index()
days_since_prior_order_by_prouse.columns = ['user_id','product_id','days_since_prior_order', 'new_days_since_prior_order']

###########################################################################################

prior_product_by_prouse_y = new_products__prior[['user_id','product_id','order_dow','name_of_day_time','product_name']]
new_product = new_products__prior['product_name'].values
prior_product_by_prouse_y['new_product_count'] = new_product
prior_product_by_prouse_y = prior_product_by_prouse_y.groupby(['user_id','product_id','order_dow','name_of_day_time','product_name'])["new_product_count"].aggregate("count").reset_index()
prior_product_by_prouse_y.columns = ['user_id','product_id','order_dow','name_of_day_time','product_name','new_product_count_y']

#############################################################################################

prior_product_by_prouse_z = new_products__prior[['user_id','product_id','days_since_prior_order','order_dow','name_of_day_time','product_name']]
new_product = new_products__prior['product_name'].values
prior_product_by_prouse_z['new_product_count'] = new_product
prior_product_by_prouse_z = prior_product_by_prouse_z.groupby(['user_id','product_id','days_since_prior_order','order_dow','name_of_day_time','product_name'])["new_product_count"].aggregate("count").reset_index()
prior_product_by_prouse_z.columns = ['user_id','product_id','days_since_prior_order','order_dow','name_of_day_time','product_name','new_product_count_z']

In [ ]:
#adding above given feature in train and test data
train_data = pd.merge(train_data, order_dow_by_prouse, how="left", on=['user_id','product_id','order_dow'])
train_data = pd.merge(train_data, order_hour_of_day_by_prouse, how="left", on=['user_id','product_id','order_hour_of_day'])
train_data = pd.merge(train_data, days_since_prior_order_by_prouse, how="left", on=['user_id','product_id','days_since_prior_order'])
train_data = pd.merge(train_data, prior_product_by_prouse_y, how="left", on=['user_id','product_id','order_dow','name_of_day_time','product_name'])
train_data = pd.merge(train_data, prior_product_by_prouse_z, how="left", on=['user_id','product_id','days_since_prior_order','order_dow','name_of_day_time','product_name'])

test_data = pd.merge(test_data, order_dow_by_prouse, how="left", on=['user_id','product_id','order_dow'])
test_data = pd.merge(test_data, order_hour_of_day_by_prouse, how="left", on=['user_id','product_id','order_hour_of_day'])
test_data = pd.merge(test_data, days_since_prior_order_by_prouse, how="left", on=['user_id','product_id','days_since_prior_order'])
test_data = pd.merge(test_data, prior_product_by_prouse_y, how="left", on=['user_id','product_id','order_dow','name_of_day_time','product_name'])
test_data = pd.merge(test_data, prior_product_by_prouse_z, how="left", on=['user_id','product_id','days_since_prior_order','order_dow','name_of_day_time','product_name'])


In [ ]:
'''
i am creating function which in return create dataframe in which product purchased by user in give day of week is arrange in in the form of rank.
products with lower rank indicates the product is less purchased by user whereas  higher rank indicates the product is more purchasedby user.
in this way we can give weightage to product on the basis of day of week because user found to be doing shopping more within week.

'''

def rank_by_week(day):
  orders_=orders[(orders.order_hour_of_day >= 7) & (orders.order_hour_of_day <= 20)]
  order_by_week = orders_[['order_id','user_id','order_dow']]
  order_prior = pd.merge(order_products_prior, order_by_week, how="left", on=['order_id'])
  order_prior = order_prior[['order_id','product_id','user_id','reordered','order_dow']]
  order_prior = order_prior[(order_prior['reordered'] == 1)]

  order_prior = order_prior[['order_id','product_id','user_id','order_dow']]

  order_prior_product = pd.merge(order_prior, products, how="left", on=['product_id'])

  order_prior_product = order_prior_product[['user_id','order_dow','product_name']]
  y = order_prior_product['product_name'].values
  order_prior_product['new_product'] = y
  order_prior_product = order_prior_product[(order_prior_product['order_dow'] == day)]

  # get the count of each product and number of reorders by the customer #
  order_prior_product_groupby = order_prior_product.groupby(["user_id","order_dow",'product_name'])["new_product"].aggregate(["count"]).reset_index()
  order_prior_product_groupby.columns = ["user_id", "order_dow", "product_name", "product_count"]

  j = np.array([i for i in order_prior_product_groupby['user_id'].value_counts()])

  p = np.percentile(j, 50)

  order_prior_product_groupby['rank'+str(day)] = order_prior_product_groupby.groupby(["user_id","order_dow"])['product_count'].rank(ascending=True,method='dense')

  order_prior_product_groupby = order_prior_product_groupby[(order_prior_product_groupby['rank'+str(day)] <= p)]
  order_prior_product_groupby = order_prior_product_groupby[["user_id","order_dow",'product_name','rank'+str(day)]]
  return order_prior_product_groupby

In [ ]:
order_prior_product_day0 = rank_by_week(0)
order_prior_product_day1 = rank_by_week(1)
order_prior_product_day2 = rank_by_week(2)
order_prior_product_day3 = rank_by_week(3)
order_prior_product_day4 = rank_by_week(4)
order_prior_product_day5 = rank_by_week(5)
order_prior_product_day6 = rank_by_week(6)

In [ ]:
train_data = pd.merge(train_data, order_prior_product_day0, how="left", on=["user_id","order_dow",'product_name'])
train_data = pd.merge(train_data, order_prior_product_day1, how="left", on=["user_id","order_dow",'product_name'])
train_data = pd.merge(train_data, order_prior_product_day2, how="left", on=["user_id","order_dow",'product_name'])
train_data = pd.merge(train_data, order_prior_product_day3, how="left", on=["user_id","order_dow",'product_name'])
train_data = pd.merge(train_data, order_prior_product_day4, how="left", on=["user_id","order_dow",'product_name'])
train_data = pd.merge(train_data, order_prior_product_day5, how="left", on=["user_id","order_dow",'product_name'])
train_data = pd.merge(train_data, order_prior_product_day6, how="left", on=["user_id","order_dow",'product_name'])

test_data = pd.merge(test_data, order_prior_product_day0, how="left", on=["user_id","order_dow",'product_name'])
test_data = pd.merge(test_data, order_prior_product_day1, how="left", on=["user_id","order_dow",'product_name'])
test_data = pd.merge(test_data, order_prior_product_day2, how="left", on=["user_id","order_dow",'product_name'])
test_data = pd.merge(test_data, order_prior_product_day3, how="left", on=["user_id","order_dow",'product_name'])
test_data = pd.merge(test_data, order_prior_product_day4, how="left", on=["user_id","order_dow",'product_name'])
test_data = pd.merge(test_data, order_prior_product_day5, how="left", on=["user_id","order_dow",'product_name'])
test_data = pd.merge(test_data, order_prior_product_day6, how="left", on=["user_id","order_dow",'product_name'])

In [ ]:
'''
i am capturing the time in sec from 1 day of month the particular product is purchased .

'''

train_data['time_in_sec'] = train_data['days_since_prior_order']*24*60*60 + train_df['order_hour_of_day']*60*60
test_data['time_in_sec'] = test_data['days_since_prior_order']*24*60*60 + test_df['order_hour_of_day']*60*60

In [ ]:
#creating one hot encoding of reorder_pattern and name_of_day_time
from sklearn.preprocessing import OneHotEncoder
t_train = train_data[['reorder_pattern', 'name_of_day_time']]
t_test = test_data[['reorder_pattern', 'name_of_day_time']]

pattern = OneHotEncoder(handle_unknown='ignore')

pattern.fit(t_train[['reorder_pattern']])
a_train = pattern.transform(t_train[['reorder_pattern']])
a_test = pattern.transform(t_test[['reorder_pattern']])

pattern1 = OneHotEncoder(handle_unknown='ignore')

pattern1.fit(t_train[['name_of_day_time']])
b_train = pattern1.transform(t_train[['name_of_day_time']])
b_test = pattern1.transform(t_test[['name_of_day_time']])

a_train = a_train.toarray()
a_test = a_test.toarray()
b_train = b_train.toarray()
b_test = b_test.toarray()

m_train = np.concatenate((a_train, b_train), axis=1)
hot_train = pd.DataFrame(data = m_train,  index = None,  columns = ['1','2','3','4','5','6','7','8'])

m_test = np.concatenate((a_test, b_test), axis=1)
hot_test = pd.DataFrame(data = m_test,  index = None,  columns = ['1','2','3','4','5','6','7','8'])


train_data = pd.concat([train_data, hot_train], axis=1)
test_data = pd.concat([test_data, hot_test], axis=1)

In [ ]:
'''
in training as well as test dataset , products has been associates with user on which we have to tari and test the model.
i am capturing the feature which tell us that if the particular product had been bought by the user previously same time frame. 

Ex := if in train dataset if the product P is associate with user A and given time frame is C dow , D hour , E days_since_prior_order then , i want
to know if the same product P had user A purchased in same C or D or E time frame in previous purchase history and if answer is yes , 
then 'exit' indicates 1 or if No then 0. 
'''

ord_prior = pd.merge(order_products_prior, orders, how="inner", on=['order_id'])
ord_prior_pro = pd.merge(ord_prior, products, how="inner", on=['product_id'])
use_hour_pro = ord_prior_pro[['user_id','order_hour_of_day','product_name']]
use_hour_pro['exit'] = [1]*use_hour_pro.shape[0]
use_hour_pro.to_csv('use_hour_pro.csv')

################################################################################

ord_prior = pd.merge(order_products_prior, orders, how="inner", on=['order_id'])
ord_prior_pro = pd.merge(ord_prior, products, how="inner", on=['product_id'])
use_pattern_pro = ord_prior_pro[['user_id','name_of_day_time','product_name']]
use_pattern_pro['exit'] = [1]*use_pattern_pro.shape[0]
use_pattern_pro.to_csv('use_pattern_pro.csv')


###############################################################################

ord_prior = pd.merge(order_products_prior, orders, how="inner", on=['order_id'])
ord_prior_pro = pd.merge(ord_prior, products, how="inner", on=['product_id'])
use_dow_pro = ord_prior_pro[['user_id','order_dow','product_name']]
use_dow_pro['exit'] = [1]*use_dow_pro.shape[0]
use_dow_pro.to_csv('use_dow_pro.csv')


##############################################################################
ord_prior = pd.merge(order_products_prior, orders, how="inner", on=['order_id'])
ord_prior_pro = pd.merge(ord_prior, products, how="inner", on=['product_id'])
use_days_since_pro = ord_prior_pro[['user_id','days_since_prior_order','product_name']]
use_days_since_pro['exit'] = [1]*use_days_since_pro.shape[0]
use_days_since_pro.to_csv('use_days_since_pro.csv')


In [ ]:
use_hour_pro.to_csv('use_hour_pro.csv')
use_pattern_pro.to_csv('use_pattern_pro.csv')
use_dow_pro.to_csv('use_dow_pro.csv')
use_days_since_pro.to_csv('use_days_since_pro.csv')
use_pro_count.to_csv('use_pro_count.csv')

In [ ]:
name2 = ['user_id', 'days_since_prior_order', 'product_name', 'exit']
name3 = ['user_id', 'order_hour_of_day', 'product_name', 'exit']
name4 = ['user_id', 'name_of_day_time', 'product_name', 'exit']
name5 = ['user_id', 'order_dow', 'product_name', 'exit']

use_days_since_pro = pd.read_csv('use_days_since_pro.csv', usecols= name2)
use_days_since_pro = use_days_since_pro.drop_duplicates()
use_days_since_pro.columns = ['user_id', 'days_since_prior_order', 'product_name', 'exit1']

use_hour_pro = pd.read_csv('use_hour_pro.csv', usecols= name3)
use_hour_pro = use_hour_pro.drop_duplicates()
use_hour_pro.columns = ['user_id', 'order_hour_of_day', 'product_name', 'exit2']

use_pattern_pro = pd.read_csv('use_pattern_pro.csv', usecols= name4)
use_pattern_pro = use_pattern_pro.drop_duplicates()
use_pattern_pro.columns = ['user_id', 'name_of_day_time', 'product_name', 'exit3']

use_dow_pro = pd.read_csv('use_dow_pro.csv', usecols= name5)
use_dow_pro = use_dow_pro.drop_duplicates()
use_dow_pro.columns = ['user_id', 'order_dow', 'product_name', 'exit4']


In [ ]:
train_data = pd.merge(train_data, use_days_since_pro, how="left", on=['user_id', 'days_since_prior_order', 'product_name'])
train_data = pd.merge(train_data, use_hour_pro, how="left", on=['user_id', 'order_hour_of_day', 'product_name'])
train_data = pd.merge(train_data, use_pattern_pro, how="left", on=['user_id', 'name_of_day_time', 'product_name'])
train_data = pd.merge(train_data, use_dow_pro, how="left", on=['user_id', 'order_dow', 'product_name'])

test_data = pd.merge(test_data, use_days_since_pro, how="left", on=['user_id', 'days_since_prior_order', 'product_name'])
test_data = pd.merge(test_data, use_hour_pro, how="left", on=['user_id', 'order_hour_of_day', 'product_name'])
test_data = pd.merge(test_data, use_pattern_pro, how="left", on=['user_id', 'name_of_day_time', 'product_name'])
test_data = pd.merge(test_data, use_dow_pro, how="left", on=['user_id', 'order_dow', 'product_name'])

In [ ]:
'''
since there are 3 attributes in dataframe which include product name, aisles name , department name .  i am joining 3 column together group by user id 
and then perfforming average W2V on top of that just to see if it could boost the model because user number of times purchasing item influence the
reorder pattern.  
'''
def word_concat():
  z = []
  name = ['product_name','department','aisle']
  for i in range(3):
    if i == 0:
      pro_order = pd.merge(order_products_prior, orders, how="inner", on=['order_id'])
      pro_order = pd.merge(pro_order, products, how="inner", on=['product_id'])
      user_product = pro_order[['user_id','product_name']]

    if i == 1:
      pro_order = pd.merge(order_products_prior, orders, how="inner", on=['order_id'])
      pro_order = pd.merge(pro_order, products, how="inner", on=['product_id'])
      pro_order = pd.merge(pro_order, departments, how="inner", on=['department_id'])
      user_product = pro_order[['user_id','department']]

    if i == 2:
      pro_order = pd.merge(order_products_prior, orders, how="inner", on=['order_id'])
      pro_order = pd.merge(pro_order, products, how="inner", on=['product_id'])
      pro_order = pd.merge(pro_order, aisles, how="inner", on=['aisle_id'])
      user_product = pro_order[['user_id','aisle']]

    user_unique = list(set(user_product['user_id'].values))  

    product_id_name = []
    for j in user_unique:
      li = []
      k = user_product[(user_product['user_id'] == j)]
      li = list(k[name[i]].values)
      product_id_name.append(li) 

    product_id_name = [' '.join("".join(e.split()) for e in k)  for k in product_id_name] 
    z.append(product_id_name)


  data = {'user_id':user_unique, 'sentence0':z[0],'sentence1':z[1],'sentence2':z[2]} 
  sen = pd.DataFrame(data)
  sen['all_sentence'] = sen['sentence0'] +' '+ sen['sentence1'] +' '+ sen['sentence2']
  sen = sen[['user_id','all_sentence']]

  return sen

sen = word_concat()  

In [ ]:
'''
i am doing Average W2V instead of bag of wof or Tfidf because w2v can help us predict the similar product,  aisles and department name.

'''

# Training our own Word2Vec model using your own text corpus
i=0
list_sentance=[]
for sentance in sen['all_sentence'].values:
    list_sentance.append(sentance.split())

model = gensim.models.Word2Vec(list_of_sentance, size=50, min_count=1, workers=4)



In [ ]:
new_data_train = train_data.replace(' ','',regex=True)
new_data_train['all_sen'] = new_data_train['department'] +' '+ new_data_train['aisle'] + ' ' +new_data_train['product_name']

new_data_test = test_data.replace(' ','',regex=True)
new_data_test['all_sen'] = new_data_test['department'] +' '+ new_data_test['aisle'] + ' ' +new_data_test['product_name']

In [ ]:
#train dataset
i=0
list_sentance=[]
for sentance in new_data_train['all_sen'].values:
    list_sentance.append(sentance.split())


sentence_vector_train = []
for sentence in list_sentance:
  sent_vectors = np.zeros(5)
  count = 0
  for word in sentence:
    if word in model:
      vector = model.wv[word]
      sent_vectors += vector
      count += 1
  if count != 0:
    sent_vectors /= count
  sentence_vector_train.append(sent_vectors) 

In [ ]:
#test dataset
i=0
list_sentance=[]
for sentance in new_data_test['all_sen'].values:
    list_sentance.append(sentance.split())

sentence_vector_test = []
for sentence in list_sentance:
  sent_vectors = np.zeros(5)
  count = 0
  for word in sentence:
    if word in model:
      vector = model.wv[word]
      sent_vectors += vector
      count += 1
  if count != 0:
    sent_vectors /= count
  sentence_vector_test.append(sent_vectors) 


In [ ]:
w2v_name = ['B'+str(i) for i in range(5)]
w2v_train = pd.DataFrame(data = sentence_vector_train,  index = None,  columns = w2v_name)

w2v_test = pd.DataFrame(data = sentence_vector_test,  index = None,  columns = w2v_name)

In [ ]:
train_data = pd.concat([train_data,w2v_train], axis=1)
test_data = pd.concat([test_data,w2v_test], axis=1)

In [ ]:
'''
in this section , i am creating feature regarding how much percentage of particular product is reorder by user in particular 
day of week or hour of day or day of month. it is carried out for Active customer which mean from the graph , on every 
weekend i.e 7 , 14, 21 and 28 , there are some user who wish to buy their products for whole week.


'''


li = [['user_id','order_dow','product_name'],['user_id','order_hour_of_day','product_name'],['user_id','days_since_prior_order','product_name'],
      ['user_id','reorder_pattern','product_name'],['user_id','name_of_day_time','product_name'],['user_id','order_dow','order_hour_of_day','product_name'],
     ['user_id','order_dow','days_since_prior_order','product_name'],['user_id','reorder_pattern','order_hour_of_day','product_name'],
      ['user_id','name_of_day_time','days_since_prior_order','product_name'],['user_id','order_dow','order_hour_of_day','days_since_prior_order','product_name',]]

merged_data = prior_data[(prior_data['days_since_prior_order'] == 7) | (prior_data['days_since_prior_order'] == 14) | 
                (prior_data['days_since_prior_order'] == 21) | (prior_data['days_since_prior_order'] == 28) ]

for i in range(10):
  a = merged_data.groupby(li[i])["new_product"].aggregate("count").reset_index()
  b = a.groupby(li[i][:-1])["new_product"].apply(lambda x:  x / x.sum())
  a['per'+str(i)] = b.to_list()
  c = a[li[i]+['per'+str(i)]]
  new_train = pd.merge(new_train,c,on=li[i],how = 'left')
  new_test = pd.merge(new_test,c,on=li[i],how = 'left')

In [ ]:
# doing one hot encoding for 'reorder_pattern' and 'name_of_day_time'
from sklearn.preprocessing import OneHotEncoder
t_train = train_data[['reorder_pattern', 'name_of_day_time']]
t_test = test_data[['reorder_pattern', 'name_of_day_time']]

pattern = OneHotEncoder(handle_unknown='ignore')

pattern.fit(t_train[['reorder_pattern']])
a_train = pattern.transform(t_train[['reorder_pattern']])
a_test = pattern.transform(t_test[['reorder_pattern']])

pattern1 = OneHotEncoder(handle_unknown='ignore')

pattern1.fit(t_train[['name_of_day_time']])
b_train = pattern1.transform(t_train[['name_of_day_time']])
b_test = pattern1.transform(t_test[['name_of_day_time']])

a_train = a_train.toarray()
a_test = a_test.toarray()
b_train = b_train.toarray()
b_test = b_test.toarray()

m_train = np.concatenate((a_train, b_train), axis=1)
hot_train = pd.DataFrame(data = m_train,  index = None,  columns = ['1','2','3','4','5','6','7','8'])

m_test = np.concatenate((a_test, b_test), axis=1)
hot_test = pd.DataFrame(data = m_test,  index = None,  columns = ['1','2','3','4','5','6','7','8'])


train_data = pd.concat([train_data, hot_train], axis=1)
test_data = pd.concat([test_data, hot_test], axis=1)

In [ ]:
#creating feature which tell us how many time particular product has been reorder in last 5 order by user
train_reorder_status_5 = train_data['reordered_latest1'].fillna(0) + train_data['reordered_latest2'].fillna(0)+ train_data['reordered_latest3'].fillna(0) + \
                   train_data['reordered_latest4'].fillna(0) + train_data['reordered_latest5'].fillna(0)

train_data['reorder_status_5'] = train_reorder_status_5   

test_reorder_status_5 = test_data['reordered_latest1'].fillna(0) + test_data['reordered_latest2'].fillna(0)+ test_data['reordered_latest3'].fillna(0) + \
                   test_data['reordered_latest4'].fillna(0) + test_data['reordered_latest5'].fillna(0)

test_data['reorder_status_5'] = test_reorder_status_5 

In [ ]:
train = pd.read_csv("order_products__train.csv", usecols=["order_id", "product_id", "reordered"])
train = pd.merge(train, orders, how="inner", on="order_id")
train = train[["user_id", "product_id", "reordered"]]
train_data = pd.merge(train_data, train, how="left", on=["user_id", "product_id"])
train_data["reordered"].fillna(0)

In [ ]:
Y =train_df['reordered']
Y.to_csv('Y.csv')
train_data.to_csv('train_data.csv')
test_data.to_csv('test_data.csv')